In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
tf.keras.backend.clear_session()

In [5]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [6]:
subject = 'Amazonas - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [7]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_AM.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Ano,Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - IDH Longevidade,Amazonas - IDH,Amazonas - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - IDH Educacao,Amazonas - IDH Renda,Amazonas - Consumo de Cimento (t)
0,2008,5.017229e+07,2.568805e+06,12.460706,4.349579e+07,0.760020,0.703356,694.929186,5.483335e+08,2.546644e+09,...,29.241577,1002.673957,1407.801557,29.368351,23.012008,70.408467,1.658576,0.678960,0.674670,769.233000
1,2009,5.047410e+07,2.574930e+06,12.433926,4.345535e+07,0.761567,0.704503,715.226516,5.302764e+08,2.679945e+09,...,30.906682,1002.901517,1393.526887,28.853124,22.769544,71.758162,1.685334,0.682960,0.672987,744.252000
2,2010,5.169307e+07,2.569006e+06,12.457785,4.379523e+07,0.762693,0.705545,753.125620,5.046425e+08,2.891930e+09,...,35.544249,1003.401682,1311.661141,28.071885,22.643630,74.482804,1.537130,0.686918,0.671387,796.116000
3,2011,5.272696e+07,2.545977e+06,12.460974,4.398860e+07,0.763406,0.706485,768.056058,4.854622e+08,3.091988e+09,...,39.206921,1003.835072,1276.962232,27.828830,22.427653,74.624578,1.468307,0.690831,0.669868,948.011000
4,2012,5.353767e+07,2.524602e+06,12.414889,4.403561e+07,0.763641,0.707029,750.564710,4.867661e+08,3.189506e+09,...,40.620066,1004.045485,1269.809720,27.699098,22.391859,74.970536,1.457821,0.694062,0.668360,994.932000
5,2013,5.434368e+07,2.507099e+06,12.394100,4.415452e+07,0.763611,0.708365,694.726016,5.225848e+08,3.093872e+09,...,39.498004,1004.283223,1266.017495,27.731222,22.404779,74.742115,1.485513,0.699234,0.667140,1036.606000
6,2014,5.487107e+07,2.472487e+06,12.359116,4.412824e+07,0.763162,0.708895,629.906817,5.676413e+08,2.779340e+09,...,38.217511,1004.431965,1277.324904,28.055466,22.412378,73.347472,1.524622,0.702586,0.665819,771.567857
7,2015,5.527657e+07,2.446996e+06,12.336147,4.411441e+07,0.762342,0.709137,585.627424,5.179392e+08,2.561731e+09,...,38.739113,1004.517645,1275.202120,28.150076,22.434009,72.952065,1.509930,0.705552,0.664387,746.867486
8,2016,5.561997e+07,2.436680e+06,12.339032,4.416689e+07,0.770069,0.713039,527.123225,4.524144e+08,2.516859e+09,...,39.689203,1004.590070,1250.291696,27.936211,22.456139,73.785027,1.450843,0.716221,0.658519,739.457657
9,2017,5.581151e+07,2.432227e+06,12.358279,4.424802e+07,0.769075,0.713224,447.665440,4.213689e+08,2.497710e+09,...,39.521434,1004.691076,1227.047962,27.873459,22.397053,73.760599,1.416718,0.717241,0.658930,687.211000


In [8]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - IDH Longevidade,Amazonas - IDH,Amazonas - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Amazonas - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - IDH Educacao,Amazonas - IDH Renda
0,5.017229e+07,2.568805e+06,12.460706,4.349579e+07,0.760020,0.703356,694.929186,5.483335e+08,2.546644e+09,1.874883e+10,...,8.396886,29.241577,1002.673957,1407.801557,29.368351,23.012008,70.408467,1.658576,0.678960,0.674670
1,5.047410e+07,2.574930e+06,12.433926,4.345535e+07,0.761567,0.704503,715.226516,5.302764e+08,2.679945e+09,2.055015e+10,...,8.420659,30.906682,1002.901517,1393.526887,28.853124,22.769544,71.758162,1.685334,0.682960,0.672987
2,5.169307e+07,2.569006e+06,12.457785,4.379523e+07,0.762693,0.705545,753.125620,5.046425e+08,2.891930e+09,2.342247e+10,...,8.389077,35.544249,1003.401682,1311.661141,28.071885,22.643630,74.482804,1.537130,0.686918,0.671387
3,5.272696e+07,2.545977e+06,12.460974,4.398860e+07,0.763406,0.706485,768.056058,4.854622e+08,3.091988e+09,2.604265e+10,...,8.376520,39.206921,1003.835072,1276.962232,27.828830,22.427653,74.624578,1.468307,0.690831,0.669868
4,5.353767e+07,2.524602e+06,12.414889,4.403561e+07,0.763641,0.707029,750.564710,4.867661e+08,3.189506e+09,2.708753e+10,...,11.321769,40.620066,1004.045485,1269.809720,27.699098,22.391859,74.970536,1.457821,0.694062,0.668360
5,5.434368e+07,2.507099e+06,12.394100,4.415452e+07,0.763611,0.708365,694.726016,5.225848e+08,3.093872e+09,2.523394e+10,...,12.008921,39.498004,1004.283223,1266.017495,27.731222,22.404779,74.742115,1.485513,0.699234,0.667140
6,5.487107e+07,2.472487e+06,12.359116,4.412824e+07,0.763162,0.708895,629.906817,5.676413e+08,2.779340e+09,1.954361e+10,...,12.610217,38.217511,1004.431965,1277.324904,28.055466,22.412378,73.347472,1.524622,0.702586,0.665819
7,5.527657e+07,2.446996e+06,12.336147,4.411441e+07,0.762342,0.709137,585.627424,5.179392e+08,2.561731e+09,1.327596e+10,...,13.139356,38.739113,1004.517645,1275.202120,28.150076,22.434009,72.952065,1.509930,0.705552,0.664387
8,5.561997e+07,2.436680e+06,12.339032,4.416689e+07,0.770069,0.713039,527.123225,4.524144e+08,2.516859e+09,1.011370e+10,...,13.263561,39.689203,1004.590070,1250.291696,27.936211,22.456139,73.785027,1.450843,0.716221,0.658519
9,5.581151e+07,2.432227e+06,12.358279,4.424802e+07,0.769075,0.713224,447.665440,4.213689e+08,2.497710e+09,8.456309e+09,...,12.821072,39.521434,1004.691076,1227.047962,27.873459,22.397053,73.760599,1.416718,0.717241,0.658930


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      744.252000
1      796.116000
2      948.011000
3      994.932000
4     1036.606000
5      771.567857
6      746.867486
7      739.457657
8      687.211000
9      350.177000
10     533.444000
11     595.320000
12     610.155000
13     640.274000
14            NaN
Name: Amazonas - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - IDH Longevidade,Amazonas - IDH,Amazonas - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Amazonas - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - IDH Educacao,Amazonas - IDH Renda
0,-1.823984,1.303637,1.392423,-1.881765,-1.489637,-1.645646,0.602581,1.204752,-0.990234,0.166091,...,-1.373236,-2.227879,-2.010367,1.924851,2.625774,2.633383,-2.467290,1.708024,-1.621756,1.535111
1,-1.673536,1.416732,0.791547,-2.028261,-0.964358,-1.283213,0.769450,0.820657,-0.546688,0.428911,...,-1.361444,-1.801387,-1.684250,1.716762,1.560169,1.369363,-1.408300,1.984284,-1.320949,1.248581
2,-1.065906,1.307356,1.326890,-0.796982,-0.582138,-0.954158,1.081029,0.275392,0.158672,0.847995,...,-1.377109,-0.613544,-0.967461,0.523363,-0.055613,0.712946,0.729492,0.454191,-1.023245,0.976441
3,-0.550533,0.882147,1.398444,-0.096457,-0.340162,-0.657254,1.203776,-0.132596,0.824346,1.230291,...,-1.383337,0.324594,-0.346368,0.017539,-0.558307,-0.412991,0.840731,-0.256354,-0.728908,0.717851
4,-0.146413,0.487481,0.364414,0.073831,-0.260467,-0.485225,1.059975,-0.104860,1.148829,1.382743,...,0.077489,0.686550,-0.044825,-0.086726,-0.826622,-0.599598,1.112174,-0.364616,-0.485905,0.461287
5,0.255366,0.164315,-0.102045,0.504621,-0.270630,-0.063266,0.600910,0.657046,0.830615,1.112297,...,0.418312,0.399151,0.295877,-0.142008,-0.760182,-0.532243,0.932952,-0.078717,-0.096908,0.253666
6,0.518259,-0.474755,-0.886983,0.409399,-0.423086,0.104286,0.068014,1.615453,-0.215960,0.282052,...,0.716551,0.071172,0.509040,0.022826,-0.089571,-0.492626,-0.161305,0.325049,0.155273,0.028744
7,0.720395,-0.945418,-1.402348,0.359293,-0.701403,0.180503,-0.296019,0.558230,-0.940033,-0.632425,...,0.979001,0.204772,0.631829,-0.008119,0.106103,-0.379858,-0.471547,0.173370,0.378325,-0.214922
8,0.891571,-1.135894,-1.337608,0.549425,1.921611,1.413182,-0.776997,-0.835563,-1.089341,-1.093812,...,1.040606,0.448123,0.735622,-0.371251,-0.336219,-0.264487,0.182007,-0.436659,1.180811,-1.213618
9,0.987051,-1.218110,-0.905767,0.843354,1.584260,1.471729,-1.430241,-1.495939,-1.153056,-1.335633,...,0.821134,0.405152,0.880374,-0.710087,-0.466003,-0.572517,0.162840,-0.788968,1.257539,-1.143706


In [11]:
reshaped_train_input = dfToInputRNN(train_input)

In [12]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 0.85940512, -0.72744224, -0.2362819 ,  0.98046156,
          0.47860509,  0.86449885, -1.18044378, -1.0453709 ,
          1.21859064, -0.99976018,  0.79428371,  1.15575749,
         -0.52559858,  1.30260559, -0.55253046, -0.25130359,
          1.38787176,  1.47154451, -1.80649616,  1.09296291,
          0.94281138,  0.52836084,  0.61258103,  0.87656257,
          1.47801536,  1.71982086,  1.28221181,  1.39086472,
          1.58209844,  1.08872091,  0.55212962, -1.81398118,
          1.44113674,  1.11191094,  1.30470835,  0.86390516,
         -1.67365968, -1.70738939, -1.55813196, -0.48498301,
         -0.75714639,  0.40770893, -0.27394734,  2.02323219,
          1.2908951 ,  1.70465513,  1.81148084,  1.39968653,
         -1.36191442,  1.28024593,  1.11054795,  0.12662799,
          1.40414112,  1.40468999,  1.45643959,  1.58509336,
          1.32283157,  1.30222047,  1.55138602,  1.91801602,
         -0.77489784,  2.32098884, -1.1942541 , -1.17081042,
          0.66742486,  1

In [13]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0      744.252000
1      796.116000
2      948.011000
3      994.932000
4     1036.606000
5      771.567857
6      746.867486
7      739.457657
8      687.211000
9      350.177000
10     533.444000
11     595.320000
Name: Amazonas - Consumo de Cimento (t), dtype: float64

In [14]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    595.32
Name: Amazonas - Consumo de Cimento (t), dtype: float64

In [15]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - IDH Longevidade,Amazonas - IDH,Amazonas - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Amazonas - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - IDH Educacao,Amazonas - IDH Renda
12,0.548425,-0.389186,-0.34651,0.585399,-0.362055,0.429222,-1.098584,1.108405,2.635823,-0.189959,...,0.598328,0.774435,0.860523,-1.989017,-1.013503,-0.547354,1.083075,-1.613773,0.941105,-1.177883


In [16]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 0.54842464, -0.38918635, -0.34651039,  0.5853988 ,
         -0.36205544,  0.4292222 , -1.09858406,  1.10840491,
          2.63582309, -0.18995911,  0.72640489,  2.66833308,
         -0.032632  ,  0.98802435,  0.24980494, -0.08955808,
          1.85434394,  2.02706568, -2.21150703,  1.42938591,
          0.85132668,  0.14092386,  0.30902324,  1.00267904,
          2.27970137,  1.11272593,  0.61007902,  0.89064538,
          1.42231438,  1.02650562,  1.47127593, -1.53201473,
          1.73503858,  1.66610613,  1.0456894 ,  0.55289259,
         -1.81216593, -2.10342296, -2.55377478,  0.41940628,
         -1.75756445,  0.241023  , -0.98659345,  1.52184916,
          1.01398607,  1.49290967,  1.40764092,  1.87930411,
         -2.51892072,  2.83356214,  1.58620924, -0.02524128,
          1.54231542,  1.5176119 ,  1.7355269 ,  1.87941982,
          1.54198129,  1.51276712,  1.83795594,  2.62210693,
          1.04993174,  0.96476493, -1.48277861, -2.26044738,
          0.59832796,  0

In [17]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    610.155
Name: Amazonas - Consumo de Cimento (t), dtype: float64

In [18]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh')),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        batch_size=train_input.shape[0],
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [19]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [20]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3092864562, 271338828, 548800045, 4154676291, 1833438733, 266026821, 1625118914, 14583561, 561860584, 554051837, 19599144, 3477408925, 1840546885, 460846536, 3587009389, 143147933, 3260127379, 443432290, 1924078804, 1546834430, 3637665882, 2202333806, 2506247936, 348154304, 2844829888, 4121686475, 3421065320, 3457959257, 3911780046, 1324792778, 658814493, 3808875965, 3764861726, 1778377417, 1033230922, 290917278, 2524703026, 1839021322, 2502483073, 2696444984, 3449507412, 2315614140, 3177041491, 788829003, 2414611402, 3915276822, 1113159345, 2890914278, 2162845460, 3169900344, 2821908714, 608424889, 815502380, 30158694, 2065470059, 2295595433, 2942718846, 3870479523, 3125630436, 3436562842, 2782364688, 3991504347, 181380064, 2426807933, 1379698611, 483712708, 2256960641, 4269785819, 780677489, 632550751, 2358618814, 3250152718, 3340190087, 519964925, 2029913413, 4153122054, 632374517, 4218397440, 935112163, 772892031, 3633339999, 2989255187, 1874841667, 2192813711, 3399973182, 1023733

2023-09-08 14:32:02.089329: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-08 14:32:02.322402: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-08 14:32:02.322585: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

loss: 332.9533996582031
winner_seed: 3092864562


Step: 1 ___________________________________________
loss: 304.8053894042969
winner_seed: 271338828


Step: 2 ___________________________________________
loss: 521.193115234375


Step: 3 ___________________________________________
loss: 473.5435791015625


Step: 4 ___________________________________________
loss: 276.5518493652344
winner_seed: 1833438733


Step: 5 ___________________________________________
loss: 377.492919921875


Step: 6 ___________________________________________
loss: 342.8377990722656


Step: 7 ___________________________________________
loss: 236.19903564453125
winner_seed: 14583561


Step: 8 ___________________________________________
loss: 367.2384033203125


Step: 9 ___________________________________________
loss: 472.3707275390625


Step: 10 ___________________________________________
loss: 197.77008056640625
winner_seed: 19599144


Step: 11 ___________________________________________
loss: 588.5771484375


St

In [21]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 3s 3s/step - loss: 590408.2500
Epoch 2/10000
1/1 [==============================] - 0s 4ms/step - loss: 577676.3750
Epoch 3/10000
1/1 [==============================] - 0s 4ms/step - loss: 539911.3125
Epoch 4/10000
1/1 [==============================] - 0s 4ms/step - loss: 284170.9375
Epoch 5/10000
1/1 [==============================] - 0s 4ms/step - loss: 36240.6523
Epoch 6/10000
1/1 [==============================] - 0s 3ms/step - loss: 32390.9785
Epoch 7/10000
1/1 [==============================] - 0s 4ms/step - loss: 23118.3223
Epoch 8/10000
1/1 [==============================] - 0s 4ms/step - loss: 20994.2031
Epoch 9/10000
1/1 [==============================] - 0s 4ms/step - loss: 13656.5508
Epoch 10/10000
1/1 [==============================] - 0s 4ms/step - loss: 12879.8516
Epoch 11/10000
1/1 [==============================] - 0s 4ms/step - loss: 13484.5596
Epoch 12/10000
1/1 [==============================] - 0s 4ms/step - los

1/1 [==============================] - 0s 4ms/step - loss: 1342.3385
Epoch 99/10000
1/1 [==============================] - 0s 4ms/step - loss: 3031.2727
Epoch 100/10000
1/1 [==============================] - 0s 4ms/step - loss: 1621.3068
Epoch 101/10000
1/1 [==============================] - 0s 4ms/step - loss: 1535.4608
Epoch 102/10000
1/1 [==============================] - 0s 4ms/step - loss: 2279.3408
Epoch 103/10000
1/1 [==============================] - 0s 4ms/step - loss: 2818.6921
Epoch 104/10000
1/1 [==============================] - 0s 4ms/step - loss: 2463.0247
Epoch 105/10000
1/1 [==============================] - 0s 4ms/step - loss: 1608.1437
Epoch 106/10000
1/1 [==============================] - 0s 4ms/step - loss: 2397.2822
Epoch 107/10000
1/1 [==============================] - 0s 4ms/step - loss: 2279.1467
Epoch 108/10000
1/1 [==============================] - 0s 4ms/step - loss: 2342.6250
Epoch 109/10000
1/1 [==============================] - 0s 4ms/step - loss: 1180.92

1/1 [==============================] - 0s 4ms/step - loss: 2351.8894
Epoch 195/10000
1/1 [==============================] - 0s 4ms/step - loss: 1485.3580
Epoch 196/10000
1/1 [==============================] - 0s 4ms/step - loss: 2361.4011
Epoch 197/10000
1/1 [==============================] - 0s 4ms/step - loss: 1301.6796
Epoch 198/10000
1/1 [==============================] - 0s 4ms/step - loss: 2574.3806
Epoch 199/10000
1/1 [==============================] - 0s 4ms/step - loss: 1495.6124
Epoch 200/10000
1/1 [==============================] - 0s 3ms/step - loss: 2753.3066
Epoch 201/10000
1/1 [==============================] - 0s 4ms/step - loss: 3800.3083
Epoch 202/10000
1/1 [==============================] - 0s 3ms/step - loss: 2188.7551
Epoch 203/10000
1/1 [==============================] - 0s 4ms/step - loss: 2852.4695
Epoch 204/10000
1/1 [==============================] - 0s 4ms/step - loss: 996.2910
Epoch 205/10000
1/1 [==============================] - 0s 4ms/step - loss: 1897.14

1/1 [==============================] - 0s 5ms/step - loss: 2264.3533
Epoch 291/10000
1/1 [==============================] - 0s 4ms/step - loss: 3573.2441
Epoch 292/10000
1/1 [==============================] - 0s 4ms/step - loss: 1492.0405
Epoch 293/10000
1/1 [==============================] - 0s 4ms/step - loss: 2984.6809
Epoch 294/10000
1/1 [==============================] - 0s 4ms/step - loss: 2529.4089
Epoch 295/10000
1/1 [==============================] - 0s 4ms/step - loss: 1451.7568
Epoch 296/10000
1/1 [==============================] - 0s 4ms/step - loss: 897.2625
Epoch 297/10000
1/1 [==============================] - 0s 4ms/step - loss: 2992.1777
Epoch 298/10000
1/1 [==============================] - 0s 4ms/step - loss: 1221.8160
Epoch 299/10000
1/1 [==============================] - 0s 4ms/step - loss: 966.5851
Epoch 300/10000
1/1 [==============================] - 0s 4ms/step - loss: 1371.7567
Epoch 301/10000
1/1 [==============================] - 0s 4ms/step - loss: 1649.119

1/1 [==============================] - 0s 4ms/step - loss: 2146.0247
Epoch 387/10000
1/1 [==============================] - 0s 4ms/step - loss: 937.6621
Epoch 388/10000
1/1 [==============================] - 0s 4ms/step - loss: 1876.7396
Epoch 389/10000
1/1 [==============================] - 0s 3ms/step - loss: 2039.7681
Epoch 390/10000
1/1 [==============================] - 0s 4ms/step - loss: 1898.3456
Epoch 391/10000
1/1 [==============================] - 0s 4ms/step - loss: 2110.2009
Epoch 392/10000
1/1 [==============================] - 0s 3ms/step - loss: 1506.5254
Epoch 393/10000
1/1 [==============================] - 0s 4ms/step - loss: 3938.5098
Epoch 394/10000
1/1 [==============================] - 0s 4ms/step - loss: 1751.8551
Epoch 395/10000
1/1 [==============================] - 0s 3ms/step - loss: 1696.1827
Epoch 396/10000
1/1 [==============================] - 0s 3ms/step - loss: 1018.6980
Epoch 397/10000
1/1 [==============================] - 0s 4ms/step - loss: 1365.61

...

1/1 [==============================] - 0s 4ms/step - loss: 1770.1476
Epoch 867/10000
1/1 [==============================] - 0s 4ms/step - loss: 1713.1664
Epoch 868/10000
1/1 [==============================] - 0s 4ms/step - loss: 1819.6825
Epoch 869/10000
1/1 [==============================] - 0s 4ms/step - loss: 1068.6283
Epoch 870/10000
1/1 [==============================] - 0s 4ms/step - loss: 1134.8998
Epoch 871/10000
1/1 [==============================] - 0s 4ms/step - loss: 434.0045
Epoch 872/10000
1/1 [==============================] - 0s 4ms/step - loss: 2494.0232
Epoch 873/10000
1/1 [==============================] - 0s 4ms/step - loss: 753.8589
Epoch 874/10000
1/1 [==============================] - 0s 4ms/step - loss: 864.7233
Epoch 875/10000
1/1 [==============================] - 0s 4ms/step - loss: 2103.3679
Epoch 876/10000
1/1 [==============================] - 0s 4ms/step - loss: 1434.9940
Epoch 877/10000
1/1 [==============================] - 0s 4ms/step - loss: 2232.2820

Epoch 963/10000
1/1 [==============================] - 0s 4ms/step - loss: 2092.3474
Epoch 964/10000
1/1 [==============================] - 0s 4ms/step - loss: 1766.7318
Epoch 965/10000
1/1 [==============================] - 0s 4ms/step - loss: 735.7715
Epoch 966/10000
1/1 [==============================] - 0s 4ms/step - loss: 1424.4701
Epoch 967/10000
1/1 [==============================] - 0s 4ms/step - loss: 1162.3057
Epoch 968/10000
1/1 [==============================] - 0s 4ms/step - loss: 1312.6465
Epoch 969/10000
1/1 [==============================] - 0s 4ms/step - loss: 2454.3933
Epoch 970/10000
1/1 [==============================] - 0s 4ms/step - loss: 1527.6644
Epoch 971/10000
1/1 [==============================] - 0s 4ms/step - loss: 1210.8142
Epoch 972/10000
1/1 [==============================] - 0s 4ms/step - loss: 1231.2233
Epoch 973/10000
1/1 [==============================] - 0s 4ms/step - loss: 1498.0096
Epoch 974/10000
1/1 [==============================] - 0s 4ms/step

1/1 [==============================] - 0s 4ms/step - loss: 993.3249
Epoch 1060/10000
1/1 [==============================] - 0s 4ms/step - loss: 278.2354
Epoch 1061/10000
1/1 [==============================] - 0s 4ms/step - loss: 1023.0467
Epoch 1062/10000
1/1 [==============================] - 0s 4ms/step - loss: 869.3229
Epoch 1063/10000
1/1 [==============================] - 0s 4ms/step - loss: 1150.8434
Epoch 1064/10000
1/1 [==============================] - 0s 4ms/step - loss: 2216.0593
Epoch 1065/10000
1/1 [==============================] - 0s 4ms/step - loss: 1707.6366
Epoch 1066/10000
1/1 [==============================] - 0s 4ms/step - loss: 949.2322
Epoch 1067/10000
1/1 [==============================] - 0s 4ms/step - loss: 1434.8306
Epoch 1068/10000
1/1 [==============================] - 0s 4ms/step - loss: 1278.9912
Epoch 1069/10000
1/1 [==============================] - 0s 4ms/step - loss: 696.2986
Epoch 1070/10000
1/1 [==============================] - 0s 4ms/step - loss: 

1/1 [==============================] - 0s 4ms/step - loss: 1897.7979
Epoch 1155/10000
1/1 [==============================] - 0s 4ms/step - loss: 1397.9496
Epoch 1156/10000
1/1 [==============================] - 0s 4ms/step - loss: 1006.3543
Epoch 1157/10000
1/1 [==============================] - 0s 4ms/step - loss: 925.6348
Epoch 1158/10000
1/1 [==============================] - 0s 4ms/step - loss: 1222.7560
Epoch 1159/10000
1/1 [==============================] - 0s 4ms/step - loss: 1429.6188
Epoch 1160/10000
1/1 [==============================] - 0s 4ms/step - loss: 1035.3677
Epoch 1161/10000
1/1 [==============================] - 0s 4ms/step - loss: 869.9805
Epoch 1162/10000
1/1 [==============================] - 0s 4ms/step - loss: 1899.5024
Epoch 1163/10000
1/1 [==============================] - 0s 4ms/step - loss: 1535.1240
Epoch 1164/10000
1/1 [==============================] - 0s 4ms/step - loss: 1241.5027
Epoch 1165/10000
1/1 [==============================] - 0s 4ms/step - los

1/1 [==============================] - 0s 4ms/step - loss: 1414.1982
Epoch 1250/10000
1/1 [==============================] - 0s 3ms/step - loss: 493.4148
Epoch 1251/10000
1/1 [==============================] - 0s 4ms/step - loss: 681.4719
Epoch 1252/10000
1/1 [==============================] - 0s 4ms/step - loss: 285.8111
Epoch 1253/10000
1/1 [==============================] - 0s 3ms/step - loss: 1223.7393
Epoch 1254/10000
1/1 [==============================] - 0s 4ms/step - loss: 724.0027
Epoch 1255/10000
1/1 [==============================] - 0s 4ms/step - loss: 1143.2179
Epoch 1256/10000
1/1 [==============================] - 0s 4ms/step - loss: 1003.3892
Epoch 1257/10000
1/1 [==============================] - 0s 4ms/step - loss: 3104.5320
Epoch 1258/10000
1/1 [==============================] - 0s 4ms/step - loss: 1044.9337
Epoch 1259/10000
1/1 [==============================] - 0s 4ms/step - loss: 1250.5375
Epoch 1260/10000
1/1 [==============================] - 0s 4ms/step - loss:

In [22]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 595ms/step
[610.155](test_target) - [601.41876](prediction) = 8.736237792968723


In [23]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.01431806310358634

In [24]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [25]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - IDH Longevidade,Amazonas - IDH,Amazonas - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Amazonas - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - IDH Educacao,Amazonas - IDH Renda
0,-0.924122,-0.742263,0.825349,-0.569521,-1.283796,-1.243968,-1.084852,1.148205,-1.122043,-1.121418,...,-0.396211,-0.995947,-1.047594,0.868878,1.133056,1.331872,-1.066913,0.489423,-1.226856,1.235124
1,-0.465030,1.413621,-1.407209,-0.836281,0.128184,0.039396,-0.243272,0.140893,-0.184474,-0.185492,...,1.373802,-0.371548,-0.298948,0.531888,0.166368,-0.254125,-0.270454,0.904353,0.004233,-0.021028
2,1.389153,-0.671358,0.581860,1.405802,1.155611,1.204571,1.328124,-1.289098,1.306518,1.306910,...,-0.977591,1.367495,1.346542,-1.400766,-1.299425,-1.077747,1.337367,-1.393776,1.222623,-1.214095


train_target:


,Amazonas - Consumo de Cimento (t)
0,744.252
1,796.116
2,948.011


test_input:


...

,Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - IDH Longevidade,Amazonas - IDH,Amazonas - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Amazonas - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - IDH Educacao,Amazonas - IDH Renda
12,0.548425,-0.389186,-0.34651,0.585399,-0.362055,0.429222,-1.098584,1.108405,2.635823,-0.189959,...,0.598328,0.774435,0.860523,-1.989017,-1.013503,-0.547354,1.083075,-1.613773,0.941105,-1.177883


test_target:


,Amazonas - Consumo de Cimento (t)
12,610.155


1/1 [==============================] - 1s 553ms/step
Error: 8.736237792968723




[948.5446166992188,
 985.4579467773438,
 165.4776611328125,
 742.6812744140625,
 754.7125854492188,
 726.4732055664062,
 691.4413452148438,
 338.4627990722656,
 532.1251220703125,
 601.4187622070312]

In [26]:
display(targets)
display(predictions)

[994.932,
 1036.606,
 771.5678571428572,
 746.8674857142857,
 739.4576571428571,
 687.211,
 350.177,
 533.444,
 595.32,
 610.155]

[948.5446166992188,
 985.4579467773438,
 165.4776611328125,
 742.6812744140625,
 754.7125854492188,
 726.4732055664062,
 691.4413452148438,
 338.4627990722656,
 532.1251220703125,
 601.4187622070312]

In [27]:
mae = mean_absolute_error(predictions, targets)
mae

137.05056395717077

In [28]:
porcentage = mae/np.mean(targets)
porcentage

0.19396496722234927